**Daphné Hbek, Elias Rhouzlane.**
Université Pierre Marie Curie, Paris, France


---
L'objectif est de représenter les mots-croisés comme un problème de satisfaction de contraintes (CSP) pour ensuite trouver un ensemble d'instanciations (variable = valeur) qui satisfasse l'ensemble des contraintes.

# 1 Modélisation par un CSP et résolution

## 1.1 Proposition 

Un problème de satisfaction de contraintes peut être représenté par un triplet (X, D, C) où :

- $X = \{v_1, v_2, \dots, v_n\}$, est un ensemble de $n$ variables
- $D = \{d_1, d_2, \dots, d_n\}$, est l'ensemble des $n$ domaines finis associés aux variables
- $C = \{c_1, c_2, \dots, c_m\}$, est un ensemble de $m$ contraintes

Dans le cas des mots-croisés, on peut définir les différents mots à trouver comme les variables du CSP et définir les contraintes sur et entre ces mots. Ainsi, on propose d'associer une variable à chaque mot de la grille.

Un mot est une variable contrainte par une certaine taille et où les lettres doivent être les même que tout autre mot de la grille qu'il intersecte. Le domaine de chaque variable est un ensemble de mot issus d'un dictionnaire de taille fixé à l'avance. Enfin, le domaine de chaque variable est réduit aux mots du dictionnaire de même taille que le mot à trouver. Chaque lettre dans les mots est l'une des 26 de l'alphabet en plus de caractères additionnels.

La numérotation des variables dépend de l'ordre de leurs apparition dans la grille, avec en premier temps les mots horizontaux puis verticaux.

Ainsi nous avons défini notre CSP comme suivant:

- $X = \{mot_1, mot_2, \dots, mot_n\}$, est l'ensemble des mots à trouver dans la grille.
- $D = \{d_1, d_2, \dots, d_n\}$ où chaque $d_i$ est un sous-ensemble du dictionnaire associé à la variable $mot_i$
- $C = \{c_1, c_2, \dots, c_m\}$, est un ensemble de contraintes sur la taille de chaque variable et de contraintes sur l'égalité de lettre aux intersections

Un exemple de CSP suivant notre modélisation serait le suivant:

| Variables | Domaines                | Contrainte unaire | Contrainte binaire             |
|-----------|-------------------------|-------------------|--------------------------------|
| $mot_1$     | {ABLE, ACID, ..., WORM} | mot.taille = 4        | intersect(mot_1, mot_2, (2,5)) |
| $mot_2$     | {ACT, AIR, ..., YOU}    | mot.taille = 3        | None                           |
| $mot_3$     | {ACROSS, ..., WRITING}  | mot.taille = 5        | None                           |

Afin d'appliquer la contrainte supplémentaire qu'un même mot ne peut apparaitre plus d'une fois dans la grille, il suffit d'utiliser la contrainte $\text{ALL-DIFF}$ et vérifier qu'à chaque instanciation d'une variable, la valeur de l'instanciation n'a pas déjà été utilisée.

Une autre méthode, plus efficace, serait de mettre à jour le domaine des variables de taille égale à la valeur de la variable instanciée et de la supprimer de ces domaines.

## 1.2 Implémentation


Nous avons développer une classe $\text{GrilleMots}$ qui représente une instance de mots-croisés. Elle se construit à l'aide d'un fichier d'entrée au format texte (.txt) ou d'une matrice de nombre binaire (0 ou 1). L'objet construit contient la taille de la grille, le nombre de mots à trouver dans la grille ayant une taille strictement supérieur à 1 et trouve automatiquement les contraintes à satisfaire afin de résoudre ce mots-croisés.

### Exemple d'utilisation (via console)

In [ ]:
import gestDict as dic
import gestIO as io
from algorithms import Solver

La génération de la grille peut se faire aléatoirement via la méthode $\text{generegrid}$ de la classe $\text{GrilleMots}$. Par exemple, il est possible de générer une grille de taille 3 par 3 avec 0 obstacle de la manière suivante:

In [2]:
grid = io.GrilleMots.genere_grid(3, 3, 0)

On récupère le dictionnaire, le fichier source est modifiable via une variable globale dans le module $\text{gestDict}$.

In [3]:
dic.recupDictionnaire()

Récupération du dictionnaire contenu dans C:\cygwin64\home\elias\git\RP/data/Dicos/133000-mots-us.txt


Il est aussi possible de sélectionner une grille existante au format .txt et de la convertir en objet $\text{GrilleMots}$.

In [4]:
grid = io.read_file("grille1.txt")[0]
print grid


Le fichier grille1.txt existe
0 0 0 1 1
0 0 0 0 1
0 0 0 0 0
1 0 0 0 0
1 1 0 0 0


Grille 5*5 avec 10 mots



Création de l'objet utile à la résolution de notre grille de mots-croisés avec en paramètre l'objet grille et le dictionnaire.

In [5]:
solver = Solver(grid, dic.DICTIONNAIRE, random=False)

Résolution du mots-croisés par forward checking sans AC3 et avec l'heuristique Minimum-remaining-value (MRV) (3). L'objet retourné est un dictionnaire où chaque variable de mot a été instancié en respect du domaine et des contraintes.

In [6]:
solver.run(ac3=False, fc=True, heuristic=3, verbose=0)
print "Variable : Valeur"
for index_variable, valeur in solver.assignment.items():
    print "{:8} : {}".format(index_variable, valeur)

Variable : Valeur
       1 : GEE
       2 : ODIN
       3 : VADES
       4 : MERE
       5 : ROW
       6 : GOV
       7 : EDAM
       8 : EIDER
       9 : NERO
      10 : SEW


### Exemple d'utilisation (via interface)

Une interface graphique développé en PyQt est aussi disponnible, elle reprend les fonctionnalités ci-dessus et ajoute une intéractivité entre l'utilisateur et le programme.

<img src="C:\cygwin64\home\elias\git\RP\rapport\img\sc1.png">

L'interface permet d'ouvrir une grille, d'en générer une selon les paramètres de taille et du nombre d'obstacle. Elle permet de choisir les dictionnaires à utiliser pour la résolution. Il est possible de combiner différents dictionnaires. Enfin, l'inteface permet de lancer une tentative de résolution du mot-croisé. Pour cela, plusieurs paramètres sont modifiables, comme les algorithmes de résolution (FC ou CBJ) et le choix ou non d'un filtrage AC-3.

L'interface permet notamment de sauvegarder la grille avant et après résolution.

## 1.3 Algorithmes

### Heuristiques
Plusieurs heuristiques ont été mis en oeuvre:

- Minimum-remaining-value (MRV)
- Max Constraint Assignement
- Max Constraint

#### Minimum-remaining-value (MRV)
Pour le choix de la variable à instancier nous avons choisi d'utiliser une heuristique Minimum-remaining-value (MRV) qui nous retourne la variable qui a le plus petit nombre de valeur légale et issue de son domaine.

In [8]:
help(Solver.mrvHeuristic)

Help on method mrvHeuristic in module algorithms:

mrvHeuristic(self, instance, variables) unbound algorithms.Solver method
    Minimum-remaining-value (MRV) heuristic
    Variable with the smallest domain in the current assignment
    @return int



L'idée est de toujours brancher sur une variable avec le plus petit nombre de valeur légale restant (la variable dont le domaine est le plus petit en nombre de mot). 
Cette heuristique a tendance à produire des arbres maigres au sommet. Cela signifie que plusieurs variables peuvent être instanciés avec moins de nœuds recherché, et donc plus d'erreur de propagation se produisent avec un moindre coût.

#### Max Constraint Assignement
Le but est de choisir la variable qui a le plus de contraintes sur les variables déjà instanciés.

#### Max Constraint
Le but est de choisir la variable qui a le plus de contraintes.

Après observation empirique, nous avons déduit que l'heuristique la plus optimale est la Minimum-remaining-value (MRV). En effet, avec la MRV le nombre de mots testés est le plus faible avec les deux algorithmes Forward Checking (FC) et Conflict BackJumping with Forward Checking (FC-CBJ).

## Algorithme de Forward Checking (FC)

L'idée de l'algorithme Forward Checking (FC) est de garder une trace des valeurs légales des variables non assignés.

## Algorithme de Conflict BackJumping (CBJ)

# 2 Expérimentation

Des expérimentations ont été menées sur nos implémentations des algorithmes. Pour cela, nous avons appliqué nos algorithmes de résolution sur différentes instances, dont les trois grilles A, B et C, et avons enregistré les temps moyens de résolution.

Tout d'abord, nous avons évalué les performances de nos algorithmes sur les trois grilles (A, B et C) dont les résultats sont présentés dans les graphiques ci-dessous.

| Method    | Grille A | Grille B | Grille C |
|-----------|----------|----------|----------|
| AC3       | 1.06     | 2.13     | 5.17     |
| FC        | 0.00     | 0.00     | 0.00     |
| AC3 + FC | 0.00     | 0.00     | 0.00     |